# Piano to Sheet

Converting .wav piano pieces into music sheets.

I

## Pipeline break down ( beta)

1. Loading in the wave file,
2. 



## Audio Processing  Libraries

In [ ]:
pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


For the processing libraries, we will be using pretty_midi to extract out label information from the given midi files.  
And for processing our wav audio files. We will be using librosa.

## Machine Learning Lib

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import librosa 
import json
import csv
import pandas as pd
import pretty_midi as pm
import mido
import IPython.display as ipd
import torch
import torch.nn as nn
import torch.nn.functional as F
import tensorflow as tf
import math
import time

# Model Saving funtion and Model loading function

Having a model saving function and a modle loading function in which we can train the model in small epoch progress.

# Preprocessing the data

In [3]:

labels_file_path = "traindata/maestro-v3.0.0-midi/maestro-v3.0.0/maestro-v3.0.0.json"
with open(labels_file_path, 'r') as file:
    data = json.load(file)

all_sets = {}
all_sets['train'] = []
all_sets['validation'] = []
all_sets['test'] = []

def sortingsets (data, allsets):
    for key in data:
       
        if data[key] == 'train':
            all_sets['train'].append(key)
        elif data[key] == 'validation':
            all_sets['validation'].append(key)
        else:
            all_sets['test'].append(key)

def save_index_to_csv(all_sets):
    for key in all_sets:
        path = f"traindata/maestro-v3.0.0-midi/maestro-v3.0.0/{key}_indicies.csv"
        df = pd.DataFrame({'Index': all_sets[key]})
        df.to_csv(path, index=False)

def load_index_from_csv(path):
    indices = []
    with open(path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            indices.append(int(row[0]))
    return indices

def save_progress_index_to_csv(indices, epoch):
    path = f"models/training_index{epoch}.csv"
    df = pd.DataFrame({'Index': indices})
    df.to_csv(path, index=False)

def load_progress_index_from_csv(epoch):
    path = f"models/training_index{epoch}.csv"
    indices = []
    with open(path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            indices.append(int(row[0]))
    return indices

sortingsets(data['split'], all_sets)
save_index_to_csv(all_sets)




traindata/maestro-v3.0.0-midi/maestro-v3.0.0/train_indicies.csv  
traindata/maestro-v3.0.0-midi/maestro-v3.0.0/test_indicies.csv   
traindata/maestro-v3.0.0-midi/maestro-v3.0.0/validation_indicies.csv

### Data Preprocessing
implenting utility functions such as the randomizing the data set for training.

In [ ]:
### making the random seed 
np.random.seed(40)

In [4]:
## Randomizing the data set index for training purposes
def randomizeing(data_set):
    ds = np.array(data_set)
    np.random.shuffle(ds)
    return ds

## Select n indices from the givien data set
def select_N_randomized_from_set(n, data_set):
    nparry = randomizeing(data_set)
    return nparry[:n]


## ----- ----- ---------- loading function ------------------------------ ##

## function loading in the wav function
def load_wav_from_index(index):
    labels_file_path = "traindata/maestro-v3.0.0-midi/maestro-v3.0.0/maestro-v3.0.0.json"
    with open(labels_file_path, 'r') as file:
        data = json.load(file)
    wav_path = "traindata/maestro-v3.0.0/maestro-v3.0.0/" + data['audio_filename'][str(index)]
    
    return librosa.load(wav_path, sr=None)

## function loading in the midi function
def load_midi_from_index(index):
    labels_file_path = "traindata/maestro-v3.0.0-midi/maestro-v3.0.0/maestro-v3.0.0.json"
    with open(labels_file_path, 'r') as file:
        data = json.load(file)
    midi_path = "traindata/maestro-v3.0.0-midi/maestro-v3.0.0/" + data['midi_filename'][str(index)]
    return pm.PrettyMIDI(midi_path)

## ----- ----- -------- Path showing function-------------------------------- ##
    
## showing the file path audio of the wave
def show_wav_path(index):
    labels_file_path = "traindata/maestro-v3.0.0-midi/maestro-v3.0.0/maestro-v3.0.0.json"
    with open(labels_file_path, 'r') as file:
        data = json.load(file)
    wav_path = "traindata/maestro-v3.0.0/maestro-v3.0.0/" + data['audio_filename'][str(index)]
    return wav_path

## Showing the file path of the midi file of data[index]
def show_midi_path(index):
    labels_file_path = "traindata/maestro-v3.0.0-midi/maestro-v3.0.0/maestro-v3.0.0.json"
    with open(labels_file_path, 'r') as file:
        data = json.load(file)
    midi_path = "traindata/maestro-v3.0.0-midi/maestro-v3.0.0/" + data['midi_filename'][str(index)]
    return midi_path

## loading in the wav and midi pair
def load_wav_midi_pair(index): ## (wav, midi)
    return load_wav_from_index(index), load_midi_from_index(index)

In [4]:
train_data = load_index_from_csv("traindata/maestro-v3.0.0-midi/maestro-v3.0.0/train_indicies.csv")
print(len(train_data))
subset_train_data = select_N_randomized_from_set(50, train_data)

print(len(subset_train_data))

962
50


# AUDIO EXPERIMENTATION

In [ ]:
(wav, sr), midi = load_wav_midi_pair(1025)
# midi = load_midi_from_index(1025)


In [29]:
ballade1, sr = load_wav_from_index(505)

noised_ballade1 = add_gaussian_noise(ballade1, noise_level=0.01)

In [86]:
print(len(ballade1))

24082675


In [82]:
print(len(noised_ballade1))

24082675


In [ ]:
ipd.Audio(data= ballade1, rate= sr)


### Audio Preprocessing functions

#### Pretty_midi note
The MIDI object is used from the python package pretty_midi.  
Using pretty_midi range from 0 to 127. we can later tranform this into the range of 0 to 87 to match a piano

In [5]:



""" This function extracts all played notes in the midi Object, which it will be futher trained with the aligne ed wave object
    input: pm object
    It is good for debugging and seeing the midi object
"""
def extract_midi_notes(midi):
    notes = []
    for instrument in midi.instruments:
        for note in instrument.notes:
            notes.append({
                'pitch': note.pitch,
                'start': note.start,
                'end': note.end,
                'velocity': note.velocity
            })
    #preprocessing the sort
    notes.sort(key=(lambda x: x['start']))
    return notes




""" Wrting a peekable Generator for midi object"""
class PeekableGenerator:
    def __init__(self, generator):
        self._generator = generator
        self._next_item = None
        self._has_next = False
        self._advance()

    def _advance(self):
        try:
            self._next_item = self._generator.__next__()
            self._has_next = True
        except StopIteration:
            self._next_item = None
            self._has_next = False

    def peek(self):
        if not self._has_next:
            raise StopIteration("No more elements to peek at.")
        return self._next_item

    def __next__(self):
        if not self._has_next:
            raise StopIteration("No more elements.")
        current = self._next_item
        self._advance()
        return current

    def has_next(self):
        return self._has_next

    def __iter__(self):
        yield self._next_item
        self._advance()

    

"""Generator to yield midi note object at the frame during the classification
    input: pm object
"""
def midi_yielding(midi):
    all_midi_obj :list = extract_midi_notes(midi)
    ##Processing
    for note in all_midi_obj:
        yield note


""" Yielding a list of midi notes information where it fits the time frame automatically.
    Implemented using overlapping frame structure for the training.
    Begin at 0, the frame jumping at the speed of jump_len, the size of the frame is frame_len
    This function will yield the frame at the given parameter.
"""
def frame_aligning_midi(t: int, jump_len:int, frame_len: int, sr:int, midi):
    midi_generator = PeekableGenerator(midi_yielding(midi))
    midi_labels = []

    jump_time_fraction: float = jump_len * (1/sr)
    frame_time_fraction: float = frame_len * (1/sr)
    framing = [t*jump_time_fraction, t*jump_time_fraction + frame_time_fraction]
    
    last_note = 0
    while midi_generator.has_next() or last_note > framing[0]:
        
        while midi_generator.has_next() and midi_generator.peek()['start'] >= framing[0] and midi_generator.peek()['start'] < framing[1]:
            try:
                midi_labels.append(midi_generator.__next__())
            except StopIteration:
                print("Generator exhausted, no Midi Objectis being added to the label")
                break
        
        ## Yielding the list of midi notes that are fitted in side the frame
        yield midi_labels

        midi_labels.sort(key=(lambda x: x['end']))
        ##calculating the next frame time step and removing the items from the previous frame
        framing [0] += jump_time_fraction
        framing [1] += jump_time_fraction
        while len(midi_labels) > 0 and midi_labels[0]['end'] < framing[0]:
            midi_labels.pop(0)
        
        if len(midi_labels) > 0 and midi_labels[0]['end'] > framing[0]:
            last_note = midi_labels[0]['end']

    
        
  



    
""" This function returns the aligned frame at the wav data,"""
def frame_aligning_wav(t: int, jump_len: int, frame_len: int, wav):
    begin = t * jump_len
    return wav[begin: begin + frame_len]


""" This function returns the amount seconds of audio data from the wav, began on t, while using  """

def audio_segment_of(t: int, wav, seconds: float, sr: int, jump_len: int = 512, frame_len: int = 2048, ):
    size = int(seconds*sr)
    begin = t*jump_len
    
    return wav[begin: begin + size]

def audio_segment_between(begin, end, wav, sr):
    return wav[int(begin*sr): int(end*sr)]

    


### Constructing Mel Spectrogram From wav frames

In [6]:
def extract_mel_spectrogram(audio, sr, n_mels=88, hop_length=512, n_fft=4096): ##_fft is the frame length 
    """
    Extract a mel-spectrogram from raw audio.
    """
    mel_spec = librosa.feature.melspectrogram(
        y=audio, sr=sr, n_mels=n_mels, hop_length=hop_length, n_fft=n_fft
    )
    # Convert to log scale
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    return mel_spec_db





In [7]:
# Example audio input
frame_time = 2048 /sr
num_frame = 22
audio1 = audio_segment_between(0,2,ballade1, sr)
quick_sample = audio_segment_between(0, num_frame*frame_time, ballade1, sr)
mel_spectrogram = extract_mel_spectrogram(audio1, sr, n_mels =188)
print(mel_spectrogram.shape)


plt.figure(figsize=(25, 10))
librosa.display.specshow(mel_spectrogram, 
                         sr=sr, 
                         x_axis="linear")
plt.colorbar(format="%+2.f")
plt.show()
ipd.Audio(data= audio1, rate= sr)



NameError: name 'sr' is not defined

### Calculating the added noise level to the wav function.

In [8]:
def compute_noise_power(audio, snr_dB):
    """
    Compute the noise power needed for a given SNR in dB.
    :param audio: NumPy array of the audio signal.
    :param snr_dB: Desired Signal-to-Noise Ratio in dB.
    :return: Noise power.
    """
    # Calculate signal power (mean squared amplitude)
    signal_power = np.mean(audio ** 2)
    
    # Convert SNR from dB to linear scale
    snr_linear = 10 ** (snr_dB / 10)
    
    # Calculate noise power
    noise_power = signal_power / snr_linear
    return noise_power





In [9]:
def add_gaussian_noise(audio, noise_level=0.0006):
    
    noise = np.random.normal(0, noise_level, audio.shape)
    return audio + noise

def framelining (time, jump, sr):
    
    frametime = jump/sr
    print(22*frametime)



### Model for musheet


In [ ]:
class PianoNoteModel(nn.Module):
    def __init__(self, num_mel_bins=88, mel_temporal_length=89, num_frame_output=22, output_size=(88, 3)):
        """The default parameter is approximated for 1 seconds of audio data, regarding to the temporal_length"""
        super(PianoNoteModel, self).__init__()

        self.num_mel_bins = num_mel_bins
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        
        # Pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Dropout layer
        self.dropout = nn.Dropout(0.25)
        
        # Compute flattened size based on input dimensions after pooling
        # Assuming input shape is (batch_size, 1, num_mel_bins, num_frames)
        pooled_mel_bins = num_mel_bins // 2  # Adjust based on pooling
        pooled_temporal_length = mel_temporal_length // 2     # Adjust based on pooling

        
        flattened_size = pooled_mel_bins * pooled_temporal_length * 64  # Based on conv2 output channels
        
        
        # Fully connected layers
        self.fc1 = nn.Linear(flattened_size, 512)
        self.fc2 = nn.Linear(512, num_frame_output * output_size[0] * output_size[1])  # Predict for each frame
        
    def forward(self, x):
        # Input shape: (batch_size, 1, num_mel_bins, num_frames)
        
        # Convolutional layers
        
        x = F.relu(self.conv1(x))

       

        x = self.pool(F.relu(self.conv2(x)))
        
        x = self.dropout(x)
        
        # Flatten for fully connected layers
        x = x.view(x.size(0), -1)  # Flatten except batch dimension
       
        # Fully connected layers
        x = F.relu(self.fc1(x))
       
        x = self.fc2(x)
        
        # Reshape to output dimensions: (batch_size, num_frames, 88, 3)
        x = x.view(x.size(0), -1, 88, 3)
        
        return x
    

# Custom loss function, for mutipurpose loss function in the output layer
class MultiTaskLoss(nn.Module):
    def __init__(self, classification_weight=1.0, regression_weight=1.0):
        super(MultiTaskLoss, self).__init__()
        self.classification_loss = nn.CrossEntropyLoss()
        self.regression_loss = nn.MSELoss()
        self.classification_weight = classification_weight
        self.regression_weight = regression_weight

    def forward(self, classification_output, classification_target, 
                regression_output, regression_target):
        # Compute classification loss
        class_loss = self.classification_loss(classification_output, classification_target)
        
        # Compute regression loss
        reg_loss = self.regression_loss(regression_output, regression_target)
        
        # Combine with weights
        total_loss = self.classification_weight * class_loss + self.regression_weight * reg_loss
        
        return total_loss
    




### Training functions

In [11]:
from torch.utils.data import DataLoader, TensorDataset
import tqdm

## MODEL Saver
def save_model(model, optimizer, id, path = "models/"):
    """ Saving the model after each training/testing before each training progress"""
    actualPath = path + f"{id}_piano_model.pth"
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, actualPath)
    print(f"Model saved to {actualPath}")

## MODEL loader
def load_model(model, optimizer, id, path = "models/"):
    actualPath = path + f"{id}_piano_model.pth"
    checkpoint = torch.load(actualPath)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    return model, optimizer


"""This function generates the matrix label to aligne the mal_spectral gram
each matrix should consired num_frame,
poping fram_jumping after the matrix is being yield

jumping time should be
frame_jumping * (frame_len /sr)
"""
def label_generator(num_frame, frame_jumping, jump_len:int, frame_len: int, sr:int, midi):
    midi_frame_gen = PeekableGenerator(frame_aligning_midi(0, frame_len, frame_len, sr, midi)) ## Note, while using the architechture of mel_spectrogram, the we don't need to consider the jump offset in the midi side
    label = []
    counter = 0
    
    
    concur_time = 0
    concur_time_end = 0
    frame_time = frame_len /sr

    
    while midi_frame_gen.has_next():
        if (len(label) < num_frame):
            label.append(list(midi_frame_gen.__next__()))
            concur_time_end += frame_time
            
        else:
            counter+=1
            yield label
            for i in range(int(frame_jumping)):
                label.pop(0)
                concur_time += frame_time
                


"""This function tries to mimic the decayed velocity miniking the sound at which a piano has been decayed"""
def velocity_decay_sustain (velocity, onset, at_time):
    if (at_time - onset < 0.2):
        return velocity
    else:
        return math.exp((onset - at_time) * 0.6) * velocity


"""formating the label of list, in to a matrix of 88 * 3 matrix.
    each row represent a strikable key, 
    column 1 (being stricked) : 0, 1 (classification purpose)
    column 2 (onset timer) : the set of positive interger that is less than onset. (Regression purpose)
    column 3 (velocity of which is being stricked) : the set of positive integer that is less than onset. (Regression purpose) 
        note for the velocity of the piano key will be approximately alingned with a decay parameter"""
def label_formater(label, frameonset):
    ret_label = np.zeros((len(label), 88,3))

    for i in range(len(label)) :
        for key_obj in label[i]:
            pitch = key_obj['pitch'] - 20
            ret_label[i][pitch][0] = 1
            ret_label[i][pitch][1] = key_obj['start'] - frameonset if key_obj['start'] > frameonset else 0
            ret_label[i][pitch][2] = velocity_decay_sustain(key_obj['velocity'],  key_obj['start'], frameonset) 

    return ret_label




"""This function assure the input for the training will retain the dimension in the case of track is ending."""
def construct_input(spectrogram, x, y):
    if spectrogram.shape == (x,y):
        return spectrogram
    else: 
        ret = np.zeros((x,y))
        
       
        for i in range(spectrogram.shape[0]):
            ret[i][:spectrogram.shape[1]] = spectrogram[i][:spectrogram.shape[1]]
        return ret

def train_segment(wav, sr, length=1, hop=0.5):
    ret = []
    max = len(wav)/sr
    begin = 0
    while (begin < max):
        adding = length if begin + length < max else max - begin
        ret.append((begin, begin + adding))
        begin += hop

    return ret
    

def label_buffer(num_frame, label):
    if len(label) < num_frame:
        for i in range(num_frame- len(label)):
            label.append([])
    return label


""" This function will generate all the mel_spectrogram and label pair for each audio segment of the song at index"""
def generate_data_label(index, noised=True, num_frame=22, segment_jump = 0.5, frame_length = 2048, hop_length = 512):
    # fetching infomation
    (wav, sr), midi = load_wav_midi_pair(index)
    
    if noised:
        wav = add_gaussian_noise(wav)

    frame_time =  frame_length/sr
    quick_sample = audio_segment_between(0, num_frame*frame_time, wav, sr)
    x, y = extract_mel_spectrogram(quick_sample, sr).shape
    segments = train_segment(wav, sr, length = num_frame * frame_time, hop= num_frame * frame_time/2)
    
    label_gen = PeekableGenerator(label_generator(num_frame, num_frame*segment_jump, hop_length, frame_length, sr, midi))
    
    input_datas = []
    labels = []
    for beg, end in segments:
        
        if (label_gen.has_next() is False) :
            break

        audio = audio_segment_between(beg, end, wav, sr)
        mel_spectrogram = extract_mel_spectrogram(audio, sr, hop_length= hop_length, n_fft=frame_length)
        input_data = construct_input(mel_spectrogram, x,y)
        input_data = np.array(input_data)
        input_data = np.expand_dims(input_data, axis=0)

        label = label_buffer(num_frame, label_gen.__next__())
        label = label_formater(label, beg)
        label = np.array(label)

        input_datas.append(input_data)
        labels.append(label)
        
    return np.array(input_datas), np.array(labels)
   
def generate_data(index, num_frame=22, segment_jump = 0.5, frame_length = 2048, hop_length = 512):
    """This function generates the audio mel_spectrogram for the song at {index} with the defaulted training parameter
        this function is generally used for validation process.
    """
    # fetching infomation
    (wav, sr), midi = load_wav_midi_pair(index)
    wav, sr = load_wav_from_index(index)
    
    frame_time =  frame_length/sr
    quick_sample = audio_segment_between(0, num_frame*frame_time, wav, sr)
    x, y = extract_mel_spectrogram(quick_sample, sr).shape
    segments = train_segment(wav, sr, length = num_frame * frame_time, hop= num_frame * frame_time/2)
    input_datas = []
 
    for beg, end in segments:
        audio = audio_segment_between(beg, end, wav, sr)
        mel_spectrogram = extract_mel_spectrogram(audio, sr, hop_length= hop_length, n_fft=frame_length)
        input_data = construct_input(mel_spectrogram, x,y)
        input_data = np.array(input_data)
        input_data = np.expand_dims(input_data, axis=0)   
        input_datas.append(input_data)
        
    return np.array(input_datas)



def one_pass_song_train(id, index, noised=True, num_frame=22, segment_jump = 0.5, frame_length = 2048, hop_length = 512, batch_size=16):
    #load model
    """The training is based on how many frame should be trained at a time,
        the default setting is suited for the expriment set up above,
        num_frame is tried to aligned it to approx 1 second of the sample
        segment_jumping would be trying to get 50% of the sample audio
        The dimension of the input is:
        batchsize * channel * 
    """

    model = PianoNoteModel()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    #Letting the classification utility be more potent than the regression utility.
    criterion = MultiTaskLoss(classification_weight=1.7, regression_weight=0.5)
 
    
    model, optimizer = load_model(model, optimizer, id)

    # generate all data
    datas, labels = generate_data_label(index, noised=noised, num_frame=num_frame, segment_jump=segment_jump, frame_length=frame_length, hop_length=hop_length)
    datas = torch.tensor(datas, dtype=torch.float32)
    labels = torch.tensor(labels, dtype=torch.float32)
    dataset = TensorDataset(datas, labels)
    dataloader = DataLoader(dataset, batch_size = batch_size, shuffle =True)

    
    progress_bar = tqdm.tqdm(enumerate(dataloader), total=len(dataloader))
    # Let model be in training mode
    model.train()

    lossval = 0.0
    for _, (batch_inputs, batch_labels) in progress_bar:
        optimizer.zero_grad()
    
        outputs = model(batch_inputs)
        classification_output = outputs[..., 0:1]
        regression_output = outputs[..., 1:3]
        
        classification_target = batch_labels[..., 0:1]
        regression_target = batch_labels[...,1:3]
        loss = criterion.forward(classification_output, classification_target, regression_output, regression_target)

        loss.backward()
        optimizer.step()

        lossval+= loss.item()
       
        
    #save model
    save_model(model, optimizer, id)
    return lossval





def train_N_song_on_epoch (epoch, N, noised=True):
    

    tobe_done = load_progress_index_from_csv(epoch)

    ## indices to be trained for the current run
    parse_in = [tobe_done.pop() for i in range(N)]

    
    ## Passing in each index to train on one_pass_song_train
    for index in parse_in:
        one_pass_song_train()

    print("Message from train_N_song_on_epoch") 
    if (len(tobe_done) == 0):
        print(f"Epoch {epoch} training complete")
    else :
        
        print(f"Epoch {epoch} have trained {N} songs, left over songs for current epoch will be saved. Need to train {len(tobe_done)}.")
        save_progress_index_to_csv(tobe_done, epoch)





# Training

### Index Processing

Using the saved indices we have processed before  
Here are the file paths  
for training datas :traindata/maestro-v3.0.0-midi/maestro-v3.0.0/train_indicies.csv  
for test datas :traindata/maestro-v3.0.0-midi/maestro-v3.0.0/test_indicies.csv   
for validataion datas :traindata/maestro-v3.0.0-midi/maestro-v3.0.0/validation_indicies.csv  

In [141]:
## Only run this once
train_indices = load_index_from_csv("traindata/maestro-v3.0.0-midi/maestro-v3.0.0/train_indicies.csv")
validation_indices = load_index_from_csv("traindata/maestro-v3.0.0-midi/maestro-v3.0.0/validation_indicies.csv")
test_indices = load_index_from_csv("traindata/maestro-v3.0.0-midi/maestro-v3.0.0/test_indicies.csv")

def initiate_train_epoch(epoch_Number, dataset):
    train_indices = load_index_from_csv(dataset)
    train_indices = randomizeing(train_indices)
    save_progress_index_to_csv(train_indices, epoch_Number)


In [ ]:
# Initiate the training epoch. RUN ONCE BEFORE TRAINING
#initiate_train_epoch(0, "traindata/maestro-v3.0.0-midi/maestro-v3.0.0/train_indicies.csv")

# Initiate the training model. RUN ONCE BEFORE TRAINING
model = PianoNoteModel(output_size=(88, 3))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
save_model(model, optimizer, "test")

Model saved to models/test_piano_model.pth


## Experimenting repetitive training 
Experimenting training on one single sample to see if there are any progresses.

In [28]:
indicies = [686, 1272, 216, 1024, 962, 505]

In [29]:
model = PianoNoteModel(output_size=(88, 3))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
save_model(model, optimizer, "test_lr_001_e_5")

for index in indicies:
    final_loss = 0
    for i in range(3): 
        train_lost_val = one_pass_song_train("test_lr_001_e_5", index)
        print(train_lost_val)
        final_loss = train_lost_val
    print(f"final loss val for {index} is {final_loss}")



Model saved to models/test_lr_001_e_5_piano_model.pth


C:\Users\youxu\AppData\Local\Temp\ipykernel_9932\4073736929.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(actualPath)
100%|██████████| 15/15 [

Model saved to models/test_lr_001_e_5_piano_model.pth
3421.7944946289062


100%|██████████| 15/15 [00:06<00:00,  2.41it/s]


Model saved to models/test_lr_001_e_5_piano_model.pth
663.8975715637207


...

Model saved to models/test_lr_001_e_5_piano_model.pth
1548.3043251037598


100%|██████████| 67/67 [00:26<00:00,  2.52it/s]


Model saved to models/test_lr_001_e_5_piano_model.pth
1532.4745655059814


100%|██████████| 67/67 [00:27<00:00,  2.48it/s]


Model saved to models/test_lr_001_e_5_piano_model.pth
1518.1284284591675
final loss val for 505 is 1518.1284284591675


In [62]:
def progressbar():
    iterator = [i for i in range(1000)]
    for i in tqdm.tqdm(iterator):
        print(i)

(1065, 1, 88, 89)
(1065, 1, 88, 89)


### Validation functions

Using these functions to check the acuracy of our model

In [ ]:

def validation_accuracy_check(id, index, noised=True, num_frame=22, segment_jump = 0.5, frame_length = 2048, hop_length = 512, batch_size=16):
    """This function returns the accuracy of the model predicting the song at index
        arguments:
            id: trained model id
            index: index of the song.
    """
    model = PianoNoteModel()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    #Letting the classification utility be more potent than the regression utility.
    criterion = MultiTaskLoss(classification_weight=1.5, regression_weight=0.7)
 
    
    model, optimizer = load_model(model, optimizer, id)
   

    datas, labels = generate_data_label(index, num_frame=num_frame, segment_jump=segment_jump, frame_length=frame_length, hop_length=hop_length)
    

    datas = torch.tensor(datas, dtype=torch.float32)
    labels = torch.tensor(labels, dtype=torch.float32)
    dataset = TensorDataset(datas, labels)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    classification_correct = 0
    classification_true_correct = 0
    regression_error = 0

    true_classified = 0 
    num_classified = 0
    
    with torch.no_grad():
        
        for val_inputs, val_labels in dataloader:
            outputs = model(val_inputs)

            #Output separation
            classification_outputs = outputs[:,:,:, 0:1].squeeze(-1)
            regression_outputs = outputs[:, :, :, 1:3]

            #Label separation
            classification_labels = val_labels[:, :, :, 0:1].squeeze(-1)
            regression_labels = val_labels[:, :, :, 1:3]
            
            #Calculate the classification accuracy of a frame
            predicted_classes = (classification_outputs > 0.5).float()
            classification_correct += (predicted_classes == classification_labels).sum().item() / model.num_mel_bins #this should be the accuracy of a frame


            #Calculate the classification accuracy of pressed key
            pressed_key_mask = classification_labels == 1
            pressed_key_label = classification_labels[pressed_key_mask]
            pressed_key_prediction = predicted_classes[pressed_key_mask]
            true_classified += pressed_key_label.numel()
            classification_true_correct += (pressed_key_label == pressed_key_prediction).sum().item()
            

            regression_error += ((regression_outputs - regression_labels) ** 2).mean().item()
            
            num_classified += val_labels.shape[0] * val_labels.shape[1] # counting number of frames have been classified i.e. batch_number * number of frame per segment
            

    if true_classified > 0 :
        true_accuracy = classification_true_correct/true_classified
    else:
        true_accuracy = 0
    print(f"Classified {num_classified} frames of audio, classification arcuracy {classification_correct/ num_classified}, average regression error is {regression_error/ num_classified}")
    print(f"The true accuracy of classifying pressed_key are {true_accuracy}")
    return num_classified, classification_correct, regression_error


    

def model_output(id, index):
    """ This function returns the audio classification from the model {id}
        arguments : id (model id)
                    index (song index)
            return: classification result
    """
    model = PianoNoteModel()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    model, _ = load_model(model, optimizer, id)
    audio_data = generate_data(index)


    datas = torch.tensor(audio_data, dtype=torch.float32)


    dataloader = DataLoader(audio_data, batch_size = 16, shuffle =True)
    


    dataset = TensorDataset(datas, labels)
    dataloader = DataLoader(dataset, batch_size = batch_size, shuffle =True)

    
    progress_bar = tqdm.tqdm(enumerate(dataloader), total=len(dataloader))

    epoch_loss = 0.0
    for batch_inputs, batch_labels in dataloader:
        optimizer.zero_grad()
        outputs = model(batch_inputs)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    epoch_loss /= len(dataloader)




def midi_reconstruction(outputed_data):
    """Reconstruct the midi object from the outputed_data.
        Arguments:
            outputed_data: A list of outputed matrixies[batch_size * frame_number * 88 * 3]
    """





In [35]:
a, b, c = validation_accuracy_check("test_lr_001_e_5", 1)

C:\Users\youxu\AppData\Local\Temp\ipykernel_22904\1897992020.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(actualPath)


Classified 32670 frames of audio, classification arcuracy 0.9295990205081112, average regression error is 0.2041722719261396
The true accuracy of classifying pressed_key are 0.03503442496500367
